$\newcommand{\xbf}{{\bf x}}
\newcommand{\ybf}{{\bf y}}
\newcommand{\wbf}{{\bf w}}
\newcommand{\Ibf}{\mathbf{I}}
\newcommand{\Xbf}{\mathbf{X}}
\newcommand{\Rbb}{\mathbb{R}}
\newcommand{\vec}[1]{\left[\begin{array}{c}#1\end{array}\right]}
$

# Les filtres convolutifs (partie 1)
Matériel de cours rédigé par Pascal Germain, 2019
************

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
import torch
from torch import nn
torch.__version__ # Ce notebook a été conçu avec la version '1.2.0' de pytorch

### Chargement des données «MNIST»

In [ ]:
repertoire_mnist = '../data/mnist/' # Modifier le répertoire au besoin

In [ ]:
def charger_mnist(repertoire, etiquettes=None, max_par_etiquettes=None):
    if etiquettes is None:
         etiquettes = np.arange(10)
    images_list = [None] * len(etiquettes)
    labels_list = [None] * len(etiquettes)
    for i, val in enumerate(etiquettes):
        nom_fichier = repertoire + f'mnist_{val}.gz'
        images_list[i] = np.genfromtxt(nom_fichier, max_rows=max_par_etiquettes, dtype=np.float32)
        nb = images_list[i].shape[0]

        labels_list[i] = i*np.ones(nb, dtype=np.int64)
        print(val, ':', nb, 'images')
        
    x = np.vstack(images_list)
    y = np.concatenate(labels_list)
    print('Total :', len(y), 'images')
    return x, y

In [ ]:
data_x, data_y = charger_mnist(repertoire_mnist, etiquettes=None, max_par_etiquettes=1000)
data_x = data_x / 255

In [ ]:
plt.figure(figsize=(15,4))
plt.imshow(data_x, cmap=plt.cm.gray, aspect=.025)
plt.colorbar()

Nous sélectionnons aléatoirement 4 images de l'ensemble MNIST qui servirons à illustrer les couches convolutives

In [ ]:
nb_images = 4
dimension = 28
indices = np.random.randint(len(data_y), size=nb_images)
sample_x = data_x[indices,:]
sample_x.shape

Convertissons ces images en **tenseurs pyTorch** de 4 dimensions: $m \times c \times h \times l$, où:
1. $m$ est le nombre d'images (lors de l'apprentissage, cela correspondra à la taille de la «minibatch»
2. $c$ est le nombre de «canaux» de l'image. Ici, nous avons un seul canal, car les images sont en teinte de gris. Typiquement, une image  couleur aura trois canaux: rouge, vert, bleu.
3. $h$ correspond à la hauteur de l'image, en nombre de pixels.
4. $c$ correspond à la largeur de l'image, en nombre de pixels.

In [ ]:
images = torch.tensor(sample_x).view(nb_images, 1, dimension, dimension)
images.shape

Le module `torchvision` contient des fonctions utilitaires pour le traitement des images. Ici, nous utiliserons la fonction `make_grid` pour afficher plusieurs images à la fois

In [ ]:
from torchvision.utils import make_grid
def afficher_grille(images):
    plt.figure(figsize=(15,4))
    grid = make_grid(images, pad_value=torch.max(images).item())
    plt.imshow(grid[0].detach(), cmap=plt.cm.gray)
    plt.colorbar()

In [ ]:
afficher_grille(images)

### Convolutions

Soit une image $I\in\Rbb^{m\times n}$ et un filtre de convolution $F\in\Rbb^{k\times k}$
de taille impaire $k=2d+1$ (avec $d\in\mathbb N^+$).

Pour $x \in \{1+d,\ldots, m-d\}$ et $y \in \{1+d,\ldots, n-d\}$:
$$\begin{align*}
	(I\star F)[x,y] = b + \sum_{i=-d}^{+d} \sum_{j=-d}^{+d} I[x{+}i, y{+}j] \times F[i{+}d{+}1,j{+}d{+}1].
\end{align*}$$

Notons que l'équation précédente contient aussi un paramètre de biais $b\in\Rbb$.

In [ ]:
nn.Conv2d?

Nous nous concentrerons sur les 3 premiers arguments nécessaires pour la création d'un filtre convolutif:
1. Le nombre de canaux (ici 1, car notre image est en teintes de gris. Ce serait 3 pour une image en couleur),
2. Le nombre de canaux de sortie. Autrement dit, le nombre de filtres apprit (1 pour l'exemple ci-bas, mais on utilise typiquement une valeur plus élevée dans un réseau de neurones).
3. La taille de chaque filtre (en nombre de pixels).

In [ ]:
conv = nn.Conv2d(1,1,3)

In [ ]:
conv.weight

In [ ]:
conv.weight.shape

In [ ]:
conv.bias

In [ ]:
conv_images = conv(images)

In [ ]:
conv_images.shape

In [ ]:
afficher_grille(conv_images)

Notons que les images précédentes sont le résultat de l'application d'un filtre de convolution dont les poids ont été initialisés aléatoirement.

Essayons maintenant quelques *filtres de Sobel*.
Voir https://fr.wikipedia.org/wiki/Filtre_de_Sobel

In [ ]:
left_sobel =  nn.Conv2d(1,1,3, bias=False)
left_sobel.weight = nn.Parameter(torch.Tensor([[[[1, 0, -1], [2, 0, -2], [1, 0, -1]]]]))
afficher_grille(left_sobel(images))

In [ ]:
left_sobel.weight

In [ ]:
right_sobel = nn.Conv2d(1,1,3, bias=False)
right_sobel.weight = nn.Parameter(torch.Tensor([[[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]]]))
afficher_grille(right_sobel(images))

In [ ]:
up_sobel =  nn.Conv2d(1,1,3, bias=False)
up_sobel.weight = nn.Parameter(torch.Tensor([[[[1,2,1], [0, 0, 0], [-1, -2, -1]]]]))
afficher_grille(up_sobel(images))

In [ ]:
down_sobel =  nn.Conv2d(1,1,3, bias=False)
down_sobel.weight = nn.Parameter(torch.Tensor([[[[-1,-2,-1], [0, 0, 0], [1,2,1]]]]))
afficher_grille(down_sobel(images))

In [ ]:
afficher_grille(up_sobel(down_sobel(images)))

In [ ]:
afficher_grille(left_sobel(right_sobel(up_sobel(down_sobel(images)))))

### «Max Pooling»

In [ ]:
nn.MaxPool2d?

In [ ]:
mp = nn.MaxPool2d(2)
mp(images).shape

In [ ]:
afficher_grille(mp(images))

In [ ]:
afficher_grille(mp(up_sobel(images)))

In [ ]:
afficher_grille(down_sobel(mp(up_sobel(images))))

## CNN: Réseaux de neurones à convolutions

Séparons d'abord les données en un ensemble d'apprentissage et un ensemble de test

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.5, random_state=42)
print('train_x:', train_x.shape)
print('test_x:', test_x.shape)
print('train_y:', train_y.shape)
print('test_y:', test_y.shape)

In [ ]:
plt.figure(figsize=(15,2))
plt.imshow(train_x, cmap=plt.cm.gray, aspect=.025)
plt.colorbar()

Il est une bonne pratique de normaliser les valeurs des attributs avant l'apprentissage. Ci-bas, nous utilisons une fonction de *pré-traitement* de librairie `scikit-learn` qui appliquera une transformation linéaire afin que chaque attribut ait une moyenne nulle et une variance untité.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
StandardScaler?

In [ ]:
scaler = StandardScaler()
scaler.fit(train_x)
train_x_prime = scaler.transform(train_x)
test_x_prime = scaler.transform(test_x)

In [ ]:
plt.figure(figsize=(15,2))
plt.imshow(train_x_prime, vmin=-1.5, vmax=1.5, cmap=plt.cm.gray, aspect=.025)
plt.colorbar()

### Créons un réseau de neurones

Nous réutilisons la classe `reseau_classif_generique` introduite lors du TD précédent.

In [ ]:
from reseau_classif_generique import ReseauClassifGenerique

In [ ]:
ReseauClassifGenerique?

Ci-bas, nous modélisons l'architecture du réseau de neurones en créant une classe qui hérite de la classe pytorch `nn.Module`. Ce faisant, nous précisons la procédure d'initialisation (méthode `__init__`) et la fonction de propagation avant (méthode `forward`).

In [ ]:
class UneArchiPourMNIST(nn.Module):
    def __init__(self, nb_filtres=32, taille_noyau=3):
        # Initialisation de la classe de base nn.Module
        super().__init__()
        
        # Créons une couche de convolution 
        self.modele_conv = nn.Sequential(
            nn.Conv2d(1, nb_filtres, kernel_size=taille_noyau),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        # La convolution est suivie d'une couche de sortie 
        self.nb_neurones_du_milieu = nb_filtres * ((28-taille_noyau+1)//2)**2
        self.modele_plein = nn.Sequential(
            nn.Linear(self.nb_neurones_du_milieu, 10),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, x):
        # Propageons la «batch». Notez que nous devons redimensionner nos données consciencieusement
        x0 = x.view(-1, 1, 28, 28)
        x1 = self.modele_conv(x0)
        x2 = x1.view(-1, self.nb_neurones_du_milieu)
        x3 = self.modele_plein(x2)
        return x3


Comme notre classe `UneArchiPourMNIST` hérite de la classe `nn.Module`, pytorch détecte automatiquement l'ensemble des paramètres à optimiser.

In [ ]:
mon_archi = UneArchiPourMNIST(nb_filtres=1, taille_noyau=5)
mon_archi

In [ ]:
for params in mon_archi.parameters():
    print(params)
    print('----')

Dans le code suivant, l'appel à `mon_archi(une_image)` exécute `mon_archi.forward(une_image)`.

In [ ]:
une_image = torch.tensor(train_x_prime[1,:])
mon_archi(une_image)

Nous pouvons maintenant lancer la procédure d'apprentissage de notre réseau convolutif.

In [ ]:
mon_archi = UneArchiPourMNIST(nb_filtres=32, taille_noyau=3)
R = ReseauClassifGenerique(mon_archi, eta=0.1, alpha=0.1, nb_epoques=20, taille_batch=32)

In [ ]:
R.apprentissage(train_x_prime, train_y)

In [ ]:
from sklearn.metrics import accuracy_score
train_pred = R.prediction(train_x_prime)
test_pred = R.prediction(test_x_prime)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

### Visualisation des filtres appris

In [ ]:
afficher_grille(mon_archi.modele_conv[0].weight)

### Visualisation de la représentation interne du réseau

In [ ]:
def afficher_activations(x, modele, etape):
    image = torch.tensor(x).view(1,1,28,28)
    sous_modele = modele[0:etape]
    print(sous_modele)
    couche = sous_modele(image)
    afficher_grille(couche.transpose(0,1))

In [ ]:
afficher_activations(train_x_prime[0], mon_archi.modele_conv, 1)

In [ ]:
afficher_activations(train_x_prime[0], mon_archi.modele_conv, 2)

In [ ]:
afficher_activations(train_x_prime[0], mon_archi.modele_conv, 3)

************

# À vous de jouer.

Essayez d'autres architectures de réseau de neurones convolutif, et tentez d'interpréter les résultats obtenus. Parmi les différentes possibilités, vous pouvez:
* Changer la taille et le nombre de filtres convolutifs
* Ajouter une ou plusieurs couches de filtres convolutifs dans la première partie du réseau
* Ajouter davantage de couches pleinement connectées dans la seconde partie du réseau
* Comparer les résultats obtenus sans appliquer la renormalisation `StandardScaler`
